In [2]:
import sys

sys.path.append('..')
sys.path.append('../keywords/')

import importcsv

print('Importing csv file...........')
small_dataset = importcsv.opencsv('../processedspeechfinal.csv') 

Importing csv file...........


In [3]:
member_name_list = small_dataset['member_name'].unique()

In [4]:
from word_frequency_calculator import filter_dataframe
from k_decomposition import getkeywords

member_vector = {}
counter = 0
for name in member_name_list:
    counter +=1
    if counter%100 == 0:
        print(counter)
    member_speeches = filter_dataframe(small_dataset, member=name).to_list()
    keywords = getkeywords(member_speeches)
    member_vector[name] = keywords

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [5]:
#save the inverted catalog to a json file
import json

# Specify the file path
file_path = "member_ch_vectorfinal.json"

# Save the dictionary to a file
with open(file_path, 'w') as file:
    json.dump(member_vector, file)


In [ ]:
# the unique member names
names = small_dataset['member_name'].unique()
names

In [ ]:
# creates a string of the keywords list (else prob with TfidVectorizer)
words1 = []
for words in member_vector.values():
    words1.append(' '.join(words))

In [ ]:
# tfidvectorizer as usual
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(lowercase=False)
matrix = tfidvectorizer.fit_transform(words1)

In [ ]:
from heapq import nlargest

# Cosine similarity for a member against all the other members and return top k 
def find_similar(member_name, matrix, k=10):
    similarity_per_member = {}
    
    ddsim_matrix = cosine_similarity(matrix[member_name,:],matrix)
    
    for i in range(len(names)):
        if ddsim_matrix[0,i] > 0:
            similarity_per_member[i] = ddsim_matrix[0,i] 
    
    top_k_items = nlargest(k+1, similarity_per_member.items(), key=lambda x: x[1])

    return [member[0] for member in top_k_items[1:]]

In [ ]:
# create a dict with keys the members and values a list of the top k similar members 
pairs_similarity = {}

for i, member in enumerate(names):
    similar = find_similar(i,matrix)
    pairs_similarity[member] = [names[i] for i in similar]

In [ ]:
#save the inverted catalog to a json file
import json

# Specify the file path
file_path = "member_similaritiesfinal.json"

# Save the dictionary to a file
with open(file_path, 'w') as file:
    json.dump(pairs_similarity, file)